In [19]:
#import libraries
import pandas as pd
#import snowflake.connector
import numpy as np
import openai
import os
import json
from tqdm import tqdm
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

# Read openai api key from file apikey.txt
api_key_file = 'apikey-gemini.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        GOOGLE_API_KEY = f.readline()
else:
    print(f"Error: {api_key_file} not found.")
genai.configure(api_key=GOOGLE_API_KEY)

OPENAI_API_KEY = openai.api_key
content = ""

#login to snowflake db
"""con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='BOOTCAMP_WH',
                                 database='SCALE_PLAYPEN',
                                 role='SNOWFLAKE_USERS')
cs = con.cursor()
"""

"con = snowflake.connector.connect(user='vishal.kumar@scale.com',\n                                 account='pxa65918',\n                                 authenticator='externalbrowser',\n                                 warehouse='BOOTCAMP_WH',\n                                 database='SCALE_PLAYPEN',\n                                 role='SNOWFLAKE_USERS')\ncs = con.cursor()\n"

In [20]:
#List various prompt components
Prompt_Beginning = "You are an expert "
Prompt_Role = ""
Prompt_Mid = ". You are tasked with asking relevant questions about various assets, trade positions and holdings of the business. The questions you ask should be variations of following question:\n "
Prompt_Human_Question = ""
Prompt_End = "\nUsing the above information, ask a new question about the business. Do not include database terminology. Do not include direct columnn names. Ask question in natural business language. Only respond with question and nothing else, no explanation or other information."
Prompt_Column_Name = ""
Prompt_Column_Definition = ""
Prompt_Complex_Mid = ". You are tasked with asking relevant questions about various assets, trade positions and holdings of the business. Some examples of the questions you can ask are listed below\n "
Prompt_Complex_MidEnd = "You will now be provided with a few identifier-columns with their definitions and one value-column. Use one or more of identifier columns to build various types of aggregations of the value-column. You can use these aggregations: SUM, AVERAGE, COUNT, MAX, MIN, STANDARD DEVIATION, VARIANCE etc. You can also include as-of-date or day/time in the question\n"
Prompt_Temperature = 0.0

In [21]:
#Save customer provided questions in a dataframe
qdf = pd.read_csv('Customer_Questions.csv')
qdf = qdf.dropna()
qdf = qdf.reset_index(drop=True)
#print(qdf.head())

#Save all the used column names with their definitions in a dataframe
cdf = pd.read_csv('Column_Definitions.csv')
cdf = cdf.dropna()
cdf = cdf.reset_index(drop=True)
#print(cdf.head())

#Save various roles in a list called roles
roles = ['Financial Analyst', 'Portfolio Manager', 'Risk Manager', 'Trader', 'Executive']

In [22]:
Prompt_Role = np.random.choice(roles)
Prompt_Human_Question_1 = np.random.choice(qdf['Question'])
Prompt_Human_Question_2 = np.random.choice(qdf['Question'])
Prompt_Human_Question_3 = np.random.choice(qdf['Question'])
# Assign a random value between 0.1 and 0.4 to prompt_temperature variable
Prompt_Temperature = np.random.uniform(0.1, 0.4)
Identifier_Column_Name_1 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_1 = cdf[cdf['Column_Name'] == Identifier_Column_Name_1]['Column_Definition']
Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
if Identifier_Column_Name_1 == Identifier_Column_Name_2:
    Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_2 = cdf[cdf['Column_Name'] == Identifier_Column_Name_2]['Column_Definition']
Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
if Identifier_Column_Name_1 == Identifier_Column_Name_3 or Identifier_Column_Name_2 == Identifier_Column_Name_3:
    Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
Identifier_Column_Definition_3 = cdf[cdf['Column_Name'] == Identifier_Column_Name_3]['Column_Definition']
Aggregation_Column_Name = np.random.choice(cdf[cdf['Used as'] == 'Aggregation']['Column_Name'])
Aggregation_Column_Definition = cdf[cdf['Column_Name'] == Aggregation_Column_Name]['Column_Definition']

In [23]:
prompt = Prompt_Beginning + Prompt_Role + Prompt_Mid + Prompt_Human_Question + Prompt_End
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(prompt)
print(response.text)

How much revenue is projected to come from each product line in the next quarter?


In [26]:
#Create all necessary functions

#make gpt calls
def question_gemini(prompt):
    model = genai.GenerativeModel('gemini-pro')
    model.generate_content(prompt)
    response = response.text
    return response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


#combine various prompt elements for a basic prompt
def prompt_compiler_basic():
    #Choose a random role from list of roles and save it in variable Prompt_Role
    Prompt_Role = np.random.choice(roles)
    #Choose a random question from the list of questions in column called 'Question' in qdf dataframe and save it in variable Prompt_Human_Question
    Prompt_Human_Question = np.random.choice(qdf['Question'])
    #Assign a random value between 0.4 and 0.8 to prompt_temperture variable
    Prompt_Temperture = np.random.uniform(0.4,0.8)
    prompt = Prompt_Beginning + Prompt_Role + Prompt_Mid + Prompt_Human_Question + Prompt_End
    return prompt

#combine various prompt elements for a complex prompt
def prompt_compiler_complex():
    Prompt_Role = np.random.choice(roles)
    Prompt_Human_Question_1 = np.random.choice(qdf['Question'])
    Prompt_Human_Question_2 = np.random.choice(qdf['Question'])
    Prompt_Human_Question_3 = np.random.choice(qdf['Question'])
    # Assign a random value between 0.1 and 0.4 to prompt_temperature variable
    Prompt_Temperature = np.random.uniform(0.1, 0.4)
    Identifier_Column_Name_1 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_1 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_1, 'Column_Definition'].values[0]
    Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    if Identifier_Column_Name_1 == Identifier_Column_Name_2:
        Identifier_Column_Name_2 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_2 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_2, 'Column_Definition'].values[0]
    Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    if Identifier_Column_Name_1 == Identifier_Column_Name_3 or Identifier_Column_Name_2 == Identifier_Column_Name_3:
        Identifier_Column_Name_3 = np.random.choice(cdf[cdf['Used as'] == 'Identifier']['Column_Name'])
    Identifier_Column_Definition_3 = cdf.loc[cdf['Column_Name'] == Identifier_Column_Name_3, 'Column_Definition'].values[0]
    Aggregation_Column_Name = np.random.choice(cdf[cdf['Used as'] == 'Aggregation']['Column_Name'])
    Aggregation_Column_Definition = cdf.loc[cdf['Column_Name'] == Aggregation_Column_Name, 'Column_Definition'].values[0]
    prompt = Prompt_Beginning + Prompt_Role + Prompt_Complex_Mid + "\n" + Prompt_Human_Question_1 + "\n" + Prompt_Human_Question_2 + "\n" + Prompt_Human_Question_3 + "\n\n" + Prompt_Complex_MidEnd + "\nIdentifier-Column Name:" + Identifier_Column_Name_1 + " Identifier-Column Definition:" + Identifier_Column_Definition_1 + "\nIdentifier-Column Name:" + Identifier_Column_Name_2 + " Identifier-Column Definition:" + Identifier_Column_Definition_2 + "\nIdentifier-Column Name:" + Identifier_Column_Name_3 + " Identifier-Column Definition:" + Identifier_Column_Definition_3 + "\nAggregation-Column Name:" + Aggregation_Column_Name + " Aggregation-Column Definition:" + Aggregation_Column_Definition +  "\n" + Prompt_End
    return prompt

#create function prompt_cot which takes conversation_history as input and returns a response
def prompt_cot(conversation_history):
    #compile conversation history
    sc_prompt = "Read our conversation history provided below and answer the following questions:\n" + conversation_history + "\n1. Did you provide the best response? If yes, then respond 'Yes'. \nIf no, provide the improved response question and improved response only, nothing else, no apologies needed. Dont say No in response."
    sc_response = question_gemini(sc_prompt)
    complexity_prompt = "Read our conversation history provided below and answer the following questions:\n" + conversation_history + "\n2. Was the question asked by you complex enough? If yes, then respond 'Yes'. \nIf no, provide a question with higher complexity only, nothing else, no apologies needed."
    complexity_response = question_gemini(complexity_prompt)
    return sc_response, complexity_response

In [27]:
#use prompt seeding strategy to generate a response
Prompt_Modifier = "\nModify the question above to retain all of its original intentions and details, but to incorporate modified instructions below. Output just the modified question.\n"
Prompt_Complexity = "Increase the complexity in the question Output just the modified prompt.\n"
Prompt_Diversity = "Increase the diversity in the question. Output just the modified question.\n"
Prompt_Structure = "Restructure the question to make it more natural and human-like. Output just the modified question.\n"

odf = pd.DataFrame(columns=['Seed_Question', 'Iteration_1', 'Iteration_2', 'Iteration_3'])
for i in tqdm(range(25)):
    #pick the current row question from qdf and save it in Prompt_Human_Question_1
    Prompt_Human_Question_1 = qdf.loc[i, 'Question']
    #save Prompt_Human_Question_1 in Seed_Question column in row i of odf
    odf.loc[i, 'Seed_Question'] = Prompt_Human_Question_1
    #print("\n" + Prompt_Human_Question_1)
    #add another for loop for prompt amplification
    for j in range(5):
        #if j is odd then call prompt_compiler_complex function
        if j % 2 == 1:
            response = question_gemini(Prompt_Human_Question_1 + Prompt_Modifier + Prompt_Complexity)
        else:
            response = question_gemini(Prompt_Human_Question_1 + Prompt_Modifier + Prompt_Diversity)
        response = response['content']
        #save response in Iteration_+j column in row i of odf
        odf.loc[i, 'Iteration_' + str(j+1)] = response
#print(odf.head())
#save odf in a csv file
odf.to_csv('Prompt_Amplification_Output.csv')

  0%|          | 0/25 [00:01<?, ?it/s]


UnboundLocalError: local variable 'response' referenced before assignment

In [ ]:
#create an emptydf called odf with columns conversation_history, sc_response, complexity_response

#add a for loop with tqdm for 30 iterations
for i in tqdm(range(30)):
    prompt = prompt_compiler_complex()
    response = question_gemini(prompt)
    #save json key content from response in response
    response = response['content']
    conversation_history = "My question to you:\n" + prompt + "\n\n\nYour response to my question:" + response
    sc_response, complexity_response = prompt_cot(conversation_history)
    #save conversation_history, sc_response, complexity_response in odf
    odf.loc[i, 'conversation_history'] = conversation_history
    odf.loc[i, 'sc_response'] = sc_response
    odf.loc[i, 'complexity_response'] = complexity_response

#save odf in a csv file
odf.to_csv('Prompt_Randomization_Output.csv')

    